In [1]:

import torch
from nisrep.normalizing_flows.manager import *
import matplotlib.pyplot as plt
import numpy as np
torch.set_default_dtype(torch.double)

In [2]:
"""
def g(x):
    return torch.exp(-torch.sum(x**2,axis=-1)) #Vanilla Gaussian

def g(x): #ring, expect 0.51 in 2 dim
    q=torch.sqrt(torch.sum(x**2,axis=-1))
    #print(q)
    f=q.clone()
    a=torch.zeros_like(q)
    b=torch.ones_like(q)
    z=torch.where(q>0.45,a,q)
    #print(z)
    #print(torch.where(z>1,a,b))
    return torch.where(z<0.2,a,b)
"""

def g(x): #box: expect 16 in dim 2, 64 in dim 3
   
    q=torch.max(torch.abs(x),dim=-1).values
  
    a=torch.zeros_like(q)
    b=torch.ones_like(q)
   
    return torch.where(q>2,a,b)
"""
def tanp(x):
    return 100*(1+((torch.tan((x-0.5)*np.pi))**2))*np.pi   #derivative for jacobian

def gaussianp(x):
    mu=0
    sigma=1
    return torch.prod(-((x-mu)*torch.unsqueeze(torch.exp(-torch.sum((x-mu)**2/2*sigma**2,-1))/(np.sqrt(2*np.pi)*sigma**3),dim=-1)),-1)
"""
def normal(x,mu, sigma, n_flow):
    return (torch.exp(-torch.sum((x-mu)**2/(2*sigma**2),-1)))/(sigma*np.sqrt((2*np.pi)**n_flow))

n_flow=2
w = torch.empty((50000,n_flow)) 
torch.nn.init.uniform_(w)
v=100*torch.tan((w-0.5)*(np.pi))
print(v)
print("Value of the integral before mapping by CL")
print(torch.mean(torch.mul(g(v),torch.abs(torch.prod(tanp(w),axis=-1)))))
#print(torch.var(torch.mul(g(v),torch.prod(tanp(w),axis=-1))))

NF =  AffineManager(n_flow=n_flow)
NF.create_model(n_pass_through=1,n_cells=1, nn_width=2, NN=[20,20,20], roll_step=1)
#print(v)
v=NF.model(NF.format_input(v, dev=v.device))
jac=v[:,-1]
print(jac)
v=v[:,:n_flow]

mult=torch.mul(g(v),jac)

print("Value of the integral after mapping by CL")
print(torch.mean(torch.mul(mult,torch.prod(tanp(w),axis=-1))))


w = torch.empty((5000000,n_flow)) 
Z=torch.nn.init.normal_(w,std=1)

print("Value of the integral in gaussian")

print(torch.mean(g(Z)/(normal(Z,0,1,n_flow)))) 



tensor([[   83.1649, -1045.5984],
        [  -42.7809,  2609.8540],
        [   52.5258,     6.0938],
        ...,
        [ 1668.8918,  -109.2107],
        [   59.1852,  -622.4971],
        [ -556.9349,    74.9189]])
Value of the integral before mapping by CL


NameError: name 'tanp' is not defined

In [4]:
def g(x): #box: expect 0.25 in dim2, 0,0039 in dim 8, 0,00195 in dim 9
   
    q=torch.max(torch.abs(x),dim=-1).values
  
    a=torch.zeros_like(q)
    b=torch.ones_like(q)
   
    return torch.where(q>0.5,a,b)

def gs(x): #box: expect 0.25 in dim2, 0,0039 in dim 8, 0,00195 in dim 9
   
    q=torch.max(torch.abs(x),dim=-1).values
    #s=torch.min(torch.abs(x),dim=-1).values
    a=torch.zeros_like(q)
    b=torch.ones_like(q)
    ret= torch.where(q>0.5,a,b)
    sret=torch.where((q>0.6)&(q<0.65),ret,5*b)
    return sret

def gaussian(x):
    return torch.exp( -((x[:,0]-0.5)**2+(x[:,1]-0.5)**2+(x[:,2]-0.5)**2+(x[:,3]-0.5)**2)/(0.05**2)) 

def gp(x):
    #s=100*gaussian(x)
    s=(1/((x[:,0]-0.5)**2+0.001**2))
    q=(1/((x[:,1])**2+0.001**2))
    t=100*torch.ones_like(x[:,0])
    return s+t+q
gp=g    
n_flow=2
w = torch.empty((250000,n_flow)) 
torch.nn.init.uniform_(w)

print("Value of the integral before mapping by CL")
print(torch.mean(gp(w)))
print("Value of the integral before mapping by CL minus constant")
print(torch.mean(gp(w)-100))
print("Variance")
print(torch.var(gp(w)))
print("SDEV")
print(torch.std(gp(w)))
print("Assumed error")
print(torch.std(gp(w))/np.sqrt(250000))

NF =  PWQuadManager(n_flow=n_flow)
NF.create_model(n_cells=2, n_bins=10, NN=[10,10,10], dev=torch.device("cpu"))
#print(v)
v=NF.model(NF.format_input(w,torch.device("cpu")))
jac=v[:,-1]
print(jac)
v=v[:,:n_flow]

mult=torch.mul(gp(v),jac)
print("Value of the integral after mapping by CL")
print(torch.mean(mult))
optim = torch.optim.Adamax(NF._model.parameters(),lr=1e-3, weight_decay=1e-5) 
res,err=NF._train_variance_forward_seq(gp,optim, False,"./logs/tmp/",1000,200,
                                           pretty_progressbar=True,save_best=True,run=None,dev=0,
                                               integrate=True,preburn_time=50)

print("Result")
print(res)
print(err)
#print(err/np.sqrt(500))
v=NF.best_model(NF.format_input(w,torch.device("cpu")))
jac=v[:,-1]
#print(jac)
v=v[:,:n_flow]
mult=torch.mul(gp(v),jac)
print("Loss reduction")
print(NF.best_loss/NF.int_loss)
print("Variance reduction")
print(torch.var(mult)/torch.var(gp(w)))
print("Value of the integral after mapping by training")
print(torch.mean(mult))
print("Value of the integral after mapping by training new device")
print(NF.integrate(gp,25,10000))


Value of the integral before mapping by CL
tensor(0.2500)
Value of the integral before mapping by CL minus constant
tensor(-99.7500)
Variance
tensor(0.1875)
SDEV
tensor(0.4330)
Assumed error
tensor(0.0009)
tensor([1.0390, 0.6856, 1.0928,  ..., 0.5468, 0.9370, 1.2104],
       grad_fn=<SelectBackward>)
Value of the integral after mapping by CL
tensor(0.2499, grad_fn=<MeanBackward0>)


Result
0.2501900937431668
0.02491210170349214
Loss reduction
tensor(0.3823, grad_fn=<DivBackward0>)
Variance reduction
tensor(0.4614, grad_fn=<DivBackward0>)
Value of the integral after mapping by training
tensor(0.2505, grad_fn=<MeanBackward0>)
Value of the integral after mapping by training new device
----
tensor(0.2492)
tensor(0.0059)
----
tensor(0.2499)
tensor(0.0131)
(tensor(0.2499), tensor(0.0131))


In [4]:
n_flow=2
w = torch.empty((1000,n_flow)) 
torch.nn.init.uniform_(w)
maxim=torch.max(normal(w,0,1,n_flow))
i=0
res=0
q=0
while(i<5000):
    q=q+1
    w = torch.empty((1,n_flow)) 
    torch.nn.init.uniform_(w)
    w2=torch.empty((1,1))
    inter=normal(w,0,1,n_flow)
    if(inter/maxim>w2):
        res+=inter
        i=i+1
    if(i%1000==0):
        print(i)
print((res/5000)*5000/q)

print("Value of the integral before mapping by CL")
print(torch.mean(normal(w,0,1,n_flow)))

1000
1000
2000
3000
4000
5000
tensor([0.0953])
Value of the integral before mapping by CL
tensor(0.1517)


In [ ]:
w = torch.empty((120000,1)) 
torch.nn.init.uniform_(w)
z=torch.mean(1+torch.cos((w-0.5)*np.pi)**2)*np.pi#*16*(1.6)**4
print(1+torch.cos((w-0.5)*np.pi)**2)
print(z)